In [21]:
# --------- Notebook code to have the same import behavior as .py files --------------
#%reset -f
%reload_ext autoreload
%autoreload 2
import os, sys
if 'notebook_path' not in vars():
    notebook_path = os.getcwd()
os.chdir(os.path.join(notebook_path,".."))
sys.path.append(os.getcwd())
# -----------------------------------------------------------------------------------
#from src.data_modules.impulse_response import ImpulseResponseDataModule

import h5py
import lightning as L
from src.data_modules import MovingImpulseResponseDataModule
from src.cnn_model import cnn_model
#ImpulseResponseDataModule()


In [23]:
data_path = "./data/datasets/moving_dataset_small.hdf5"
sound_dir = "./data/reference_data/reference_sounds/"
data_module = MovingImpulseResponseDataModule(data_path,sound_dir,batch_size=2)

model = cnn_model()
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger(log_model="all")
trainer = L.Trainer(
    max_epochs=50,
    accelerator="cuda",
    devices=1,
    logger=wandb_logger,
)



trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: erik-tegler. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | thinker | Sequential       | 9.1 M 
2 | cnn     | Sequential       | 194 K 
---------------------------------------------
9.3 M     Trainable params
0         Non-trainable params
9.3 M     Total params
37.126    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 76/76 [00:02<00:00, 26.40it/s, v_num=qzwn]        

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 76/76 [00:03<00:00, 19.08it/s, v_num=qzwn]


In [2]:
data_path = "./data/datasets/example_moving.hdf5"
sound_dir = "./data/reference_data/reference_sounds/"
data_module = MovingImpulseResponseDataModule(data_path,sound_dir,batch_size=5)



# data_module.setup()
# for i in data_module.train_dataloader():
#     X = i[0].to("cuda")
#     break
# len(data_module.train_dataloader())
# #print(os.getcwd())

In [19]:
data_path = "./data/datasets/moving_dataset_small.hdf5"
with h5py.File(data_path, "r") as f:
    print(f["input"].shape)
    print(f.attrs.keys())


(8, 51, 10, 1600)
<KeysViewHDF5 ['fs', 'n_mics', 'n_rooms', 'reflection_coeff', 'rir_len', 'room_max_size', 'room_min_size', 'scatter_coeff', 'sound_source_locations', 'sound_source_max_move', 'speed_of_sound', 'target_path']>


In [219]:
batch = next(iter(data_module.train_dataloader()))
X,y = batch

import glob
import torch

from scipy.io import wavfile
sample_length: int = 10000 #number of samples
max_freq: float = 4000.0 # Hz
batch_size : int = 1
n_mics_per_batch = 17
max_shift : float = 500 #samples
n_shift_bins : int = 500
validation_percentage = 0.05

# remove
dataset_metadata = data_module.h5file.attrs
dataset = data_module.dataset
sound_files = glob.glob(sound_dir + "*.wav")

#const
rir_len = dataset_metadata["rir_len"]
batch_size = X.shape[0]
n_sound_positions = X.shape[2]
sim_sample_length = sample_length + rir_len # length used when simmulating sound, needs to be longer than the final sound to have echo in the entire sample


# randomly select audio pieces

signals = torch.zeros(batch_size,sim_sample_length)
for batch_i in range(batch_size):
    sound_file = sound_files[torch.randint(len(sound_files),(1,))]
    fs, signal = wavfile.read(sound_file)
    if fs != dataset_metadata["fs"]:
        raise Exception("Please use .wav with the same sampling frequency as the simmulated impulse responses sampling frequency (probably 16 kHz")
    start = torch.randint(len(signal) - sim_sample_length, (1,))
    signals[batch_i,:] = torch.tensor(signal[start : start + sim_sample_length])
piece_length = signals.shape[1] // n_sound_positions
signals = signals[:,:piece_length*n_sound_positions].reshape(batch_size, n_sound_positions, piece_length) # splitting the sound into the different speaker positions
#signals = torch.concatenate([signals, torch.zeros(signals.shape[0], signals.shape[1], rir_len)],dim=2)

signals = torch.fft.irfft(torch.fft.rfft(X, signals.shape[2] + rir_len) * torch.fft.rfft(signals, signals.shape[2] + rir_len).unsqueeze(1))

fin_signal = torch.zeros(batch_size, n_mics_per_batch, signals.shape[3] + piece_length*(n_sound_positions-1))
for i in range(n_sound_positions):
    fin_signal[:,:,i*piece_length:piece_length*i+signals.shape[3]] = signals[:,:,i]

fin_signal = fin_signal[:,:,-sample_length:]
max_freq_component = int(max_freq*sample_length/dataset_metadata["fs"])

fin_signal = torch.fft.rfft(fin_signal)[:,:,:max_freq_component].unsqueeze(2)
fin_signal = torch.concatenate([torch.concatenate([fin_signal, fin_signal.roll(i + 1, 1)], dim=2) for i in range((n_mics_per_batch-1) // 2)],dim=1)  # organize sounds pairwise, 
# NOTE: if n_mics_per_batch is not odd, then we will compute all pairs of microphones except one
fin_signal = fin_signal.view(batch_size * fin_signal.shape[1],2,fin_signal.shape[3])
fin_signal = torch.concatenate([fin_signal.real, fin_signal.imag], dim=1)

y = torch.concatenate([y - y.roll(i + 1, 1) for i in range((n_mics_per_batch-1) // 2)], dim=1).view(-1) # compute pairwise distance-difference
y *= dataset_metadata["fs"]/dataset_metadata["speed_of_sound"] # rescale distance-difference to sample_difference

bin_edges = torch.linspace(-max_shift,max_shift,n_shift_bins+1)
bin_edges[0] = -float("inf")
bin_edges[-1] = float("inf")
y = (y.unsqueeze(1) < bin_edges).to(torch.long).argmax(dim=1) - 1  # Bins the values in y, since argmax finds first occurence where condition is met.



#torch.fft.irfft(torch.fft.rfft().unsqueeze(1) * torch.fft.rfft(X, signals.shape[2]))



torch.Size([680])

In [143]:
dataset_metadata["speed_of_sound"]

343.0

In [ ]:
def format_simulated_data(X, y):
            """
            transform a tensor of impulse responses in different rooms into pairwise TimeEstimation-problems. Note (X and y should be on GPU)

            """
            # pull a random sound
            sound_paths = glob(os.path.join(reference_sound_folder, "*.wav"))
            # simulate longer sound and then cut to the relevant piece
            signals = np.zeros(
                (X.shape[0], config["sample_length"] + config["rir_len"] - 1)
            )
            for i in range(X.shape[0]):
                sound_path = sound_paths[np.random.randint(len(sound_paths))]
                fs, signal = wavfile.read(sound_path)
                start = np.random.randint(
                    0, len(signal) - config["sample_length"] - config["rir_len"] - 1
                )
                signals[i, :] = signal[
                    start : start + config["sample_length"] + config["rir_len"] - 1
                ]
            # signals = torch.tensor(signals).to(torch.float32).to(device).unsqueeze(1)
            signals = torch.tensor(signals).to(torch.float32).unsqueeze(1)

            q = torch.fft.irfft(
                torch.fft.rfft(signals) * torch.fft.rfft(X, signals.shape[2])
            )[
                :, :, : config["sample_length"]
            ]  # compute the heard sound, and cut it to the right length
            q = torch.fft.rfft(q)[
                :, :, : config["max_freq"]
            ]  # cut frequencies which are too high
            q = q.unsqueeze(2)
            q = torch.concatenate(
                [
                    torch.concatenate([q, q.roll(i + 1, 1)], dim=2)
                    for i in range(config["mics_per_batch"] // 2)
                ],
                dim=1,
            )  # organize sounds pairwise
            q = q.view(
                X.shape[0]
                * (config["mics_per_batch"] * (config["mics_per_batch"] - 1))
                // 2,
                2,
                -1,
            )  # reshape so that each example is a row
            X = torch.concatenate([q.real, q.imag], dim=1)
            X /= (
                X.std(dim=2).mean(dim=1).unsqueeze(1).unsqueeze(2) + 1e-5
            )  # avoid dividing by 0
            y = (
                torch.concatenate(
                    [
                        y - y.roll(i + 1, 1)
                        for i in range(config["mics_per_batch"] // 2)
                    ],
                    dim=1,
                ).view(-1)
                * fs
                / 343
            )  # compute gt for all pairs
            y = y_to_class_gt(y, config["guess_grid_size"], config["max_shift"]).to(
                torch.long
            )

            return X, y

In [10]:
data_module.dataset.X

<HDF5 dataset "input": shape (10, 51, 10, 1600), type "<f4">

In [4]:


with h5py.File("./test.hdf5","w") as f:
    f.create_dataset("metadata",())
    z = f["metadata"]
    #metadata = z.attrs
    f.attrs["a"] = 43
    #metadata.create("a",123)




In [5]:
from torch.utils.data import DataLoader
import torch
with h5py.File("./data/datasets/example_moving.hdf5","r") as f:
    X = f["input"]
    #torch.utils.data.IterableDataset(X)
    print(DataLoader(X))
    torch.utils.data.Subset(X,torch.arange(5))
    #metadata = z.attrs
    #for i in f.attrs:
    #    print(f'{i} : {f.attrs[i]}')
    #print(f.attrs["a"])


In [29]:
def f(x,y):
    print(locals())
    print(vars()['x'])

f(1,2)

{'x': 1, 'y': 2}
1
